# Import Libraries

In [1]:
!pip install jupyter scikit-learn tensorflow tfx flask joblib tensorflow_model_analysis==0.46.0

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 3.3 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of tfx to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.7/89.7 kB 6.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.0/152.0 kB 10.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.5/173.5 kB 11.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.9/99.9 kB 5.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━

# Set Variables

In [2]:
import os
import sys
from typing import Text
 
from absl import logging
from tfx.orchestration import metadata, pipeline
from tfx.orchestration.beam.beam_dag_runner import BeamDagRunner
 
PIPELINE_NAME = "nandaaryaputra-pipeline"
 
# pipeline inputs
DATA_ROOT = "/kaggle/input/student-depression-dataset"
COMPONENTS_FILE = "modules/components.py"
PIPELINE_FILE = "modules/pipeline.py"
TRANSFORM_MODULE_FILE = "modules/transform.py"
TUNER_MODULE_FILE = "modules/tuner.py"
TRAINER_MODULE_FILE = "modules/trainer.py"
 
# pipeline outputs
OUTPUT_BASE = "output"
serving_model_dir = os.path.join(OUTPUT_BASE, 'serving_model')
pipeline_root = os.path.join(OUTPUT_BASE, PIPELINE_NAME)
metadata_path = os.path.join(pipeline_root, "metadata.sqlite")

# Create modules package

In [3]:
!mkdir modules

In [4]:
!touch modules/__init__.py

# Generate component.py file

In [5]:
%%writefile {COMPONENTS_FILE}
"""
This module initializes the TFX pipeline components.

The components include data ingestion, data validation, feature transformation, 
hyperparameter tuning, model training, model evaluation, and model deployment. 
It builds a pipeline using TensorFlow Extended (TFX) components and integrates 
them for end-to-end machine learning workflows.

Main Components:
- CsvExampleGen: Ingests data from CSV files.
- StatisticsGen: Computes dataset statistics.
- SchemaGen: Infers the schema of the dataset.
- ExampleValidator: Detects anomalies and missing values.
- Transform: Applies feature engineering transformations.
- Tuner: Optimizes hyperparameters using a tuning module.
- Trainer: Trains a machine learning model.
- Resolver: Fetches the latest blessed model for evaluation comparison.
- Evaluator: Analyzes model performance using predefined metrics.
- Pusher: Deploys the trained model if it passes evaluation.
"""

from dataclasses import dataclass
import os
import tensorflow_model_analysis as tfma

from tfx.components import (
    CsvExampleGen, StatisticsGen, SchemaGen, ExampleValidator,
    Transform, Tuner, Trainer, Evaluator, Pusher
)
from tfx.proto import example_gen_pb2, trainer_pb2, pusher_pb2
from tfx.types import Channel
from tfx.dsl.components.common.resolver import Resolver
from tfx.types.standard_artifacts import Model, ModelBlessing
from tfx.dsl.input_resolution.strategies.latest_blessed_model_strategy import (
    LatestBlessedModelStrategy
)

@dataclass
class ComponentConfig:
    """
    Configuration class for initializing TFX components.

    Attributes:
        data_dir (str): Directory where the input data is stored.
        transform_module (str): Path to the transformation module.
        tuner_module (str): Path to the hyperparameter tuning module.
        training_module (str): Path to the model training module.
        training_steps (int): Number of steps for training.
        eval_steps (int): Number of steps for evaluation.
        serving_model_dir (str): Directory to store the trained model for deployment.
    """
    data_dir: str
    transform_module: str
    tuner_module: str
    training_module: str
    training_steps: int
    eval_steps: int
    serving_model_dir: str

def init_components(config: ComponentConfig):
    """
    Initializes TFX components required for building a pipeline for training and deploying a model.
    """
    # Disable pylint no-member as some attributes are dynamically generated 
    # and not recognized by pylint. Source: https://stackoverflow.com/a/54156144
    
    # pylint: disable=no-member
    output_config = example_gen_pb2.Output(
        split_config=example_gen_pb2.SplitConfig(
            splits=[
                example_gen_pb2.SplitConfig.Split(name='train', hash_buckets=8),
                example_gen_pb2.SplitConfig.Split(name='eval', hash_buckets=2)
            ]
        )
    ) 
    # pylint: enable=no-member
    
    example_gen = CsvExampleGen(input_base=config.data_dir, output_config=output_config)
    statistics_gen = StatisticsGen(examples=example_gen.outputs['examples'])
    schema_gen = SchemaGen(statistics=statistics_gen.outputs["statistics"])
    example_validator = ExampleValidator(statistics=statistics_gen.outputs['statistics'], schema=schema_gen.outputs['schema'])
    transform = Transform(
        examples=example_gen.outputs['examples'], 
        schema=schema_gen.outputs['schema'], 
        module_file=os.path.abspath(config.transform_module)
    )

    # pylint: disable=no-member
    tuner = Tuner(
        module_file=os.path.abspath(config.tuner_module),
        examples=transform.outputs["transformed_examples"],
        transform_graph=transform.outputs["transform_graph"],
        schema=schema_gen.outputs["schema"],
        train_args=trainer_pb2.TrainArgs(splits=["train"], num_steps=config.training_steps),
        eval_args=trainer_pb2.EvalArgs(splits=["eval"], num_steps=config.eval_steps),
    )
    # pylint: enable=no-member

    # pylint: disable=no-member
    trainer = Trainer(
        module_file=os.path.abspath(config.training_module),
        examples=transform.outputs['transformed_examples'],
        transform_graph=transform.outputs['transform_graph'],
        schema=schema_gen.outputs['schema'],
        hyperparameters=tuner.outputs["best_hyperparameters"],
        train_args=trainer_pb2.TrainArgs(splits=['train'], num_steps=config.training_steps),
        eval_args=trainer_pb2.EvalArgs(splits=['eval'], num_steps=config.eval_steps)
    )
    # pylint: enable=no-member
    
    model_resolver = Resolver(
        strategy_class=LatestBlessedModelStrategy,
        model=Channel(type=Model),
        model_blessing=Channel(type=ModelBlessing)
    ).with_id('Latest_blessed_model_resolver')
    
    eval_config = tfma.EvalConfig(
        model_specs=[tfma.ModelSpec(label_key='Depression')],
        slicing_specs=[tfma.SlicingSpec()],
        metrics_specs=[
            tfma.MetricsSpec(metrics=[
                tfma.MetricConfig(class_name='ExampleCount'),
                tfma.MetricConfig(class_name='AUC'),
                tfma.MetricConfig(class_name='FalsePositives'),
                tfma.MetricConfig(class_name='TruePositives'),
                tfma.MetricConfig(class_name='FalseNegatives'),
                tfma.MetricConfig(class_name='TrueNegatives'),
                tfma.MetricConfig(class_name='BinaryAccuracy',
                    threshold=tfma.MetricThreshold(
                        value_threshold=tfma.GenericValueThreshold(
                            lower_bound={'value': 0.5}
                        ),
                        change_threshold=tfma.GenericChangeThreshold(
                            direction=tfma.MetricDirection.HIGHER_IS_BETTER,
                            absolute={'value': 0.0001}
                        )
                    )
                )
            ])
        ]
    )
    
    evaluator = Evaluator(
        examples=example_gen.outputs['examples'],
        model=trainer.outputs['model'],
        baseline_model=model_resolver.outputs['model'],
        eval_config=eval_config
    )

    # pylint: disable=no-member
    pusher = Pusher(
        model=trainer.outputs["model"],
        model_blessing=evaluator.outputs["blessing"],
        push_destination=pusher_pb2.PushDestination(
            filesystem=pusher_pb2.PushDestination.Filesystem(
                base_directory=config.serving_model_dir
            )
        ),
    )
    # pylint: enable=no-member
    
    return (
        example_gen, statistics_gen, schema_gen, example_validator, 
        transform, tuner, trainer, model_resolver, evaluator, pusher
    )

Writing modules/components.py


# Generate pipeline.py file

In [6]:
%%writefile {PIPELINE_FILE}
"""
Pipeline definition for the student depression prediction model.

This script initializes and runs a TFX pipeline for data transformation,
hyperparameter tuning, model training, and evaluation.
"""

import os
from typing import Text

from absl import logging
from tfx.orchestration import metadata, pipeline
from tfx.orchestration.beam.beam_dag_runner import BeamDagRunner
from modules.components import ComponentConfig, init_components

PIPELINE_NAME = "nandaaryaputra-pipeline"

DATA_ROOT = "/kaggle/input/student-depression-dataset"
COMPONENTS_FILE = "modules/components.py"
PIPELINE_FILE = "modules/pipeline.py"
TRANSFORM_MODULE_FILE = "modules/transform.py"
TUNER_MODULE_FILE = "modules/tuner.py"
TRAINER_MODULE_FILE = "modules/trainer.py"

OUTPUT_BASE = "output"
serving_model_dir = os.path.join(OUTPUT_BASE, 'serving_model')
pipeline_root = os.path.join(OUTPUT_BASE, PIPELINE_NAME)
metadata_path = os.path.join(pipeline_root, "metadata.sqlite")

def init_local_pipeline(
    components_list, root_dir: Text
) -> pipeline.Pipeline:
    """
    Initialize a local TFX pipeline.

    Args:
        components_list: A list of TFX components to be included in the pipeline.
        root_dir: Root directory for pipeline output artifacts.

    Returns:
        A TFX pipeline.
    """
    logging.info(f'Pipeline root set to: {root_dir}')
    beam_args = [
        '--direct_running_mode=multi_processing',
        '--direct_num_workers=0'
    ]

    return pipeline.Pipeline(
        pipeline_name=PIPELINE_NAME,
        pipeline_root=root_dir,
        components=components_list,
        enable_cache=True,
        metadata_connection_config=metadata.sqlite_metadata_connection_config(
            metadata_path
        ),
        beam_pipeline_args=beam_args
    )


if __name__ == '__main__':
    logging.set_verbosity(logging.INFO)

    config = ComponentConfig(
        data_dir=DATA_ROOT,
        training_module=TRAINER_MODULE_FILE,
        transform_module=TRANSFORM_MODULE_FILE,
        tuner_module=TUNER_MODULE_FILE,
        training_steps=800,
        eval_steps=400,
        serving_model_dir=serving_model_dir
    )

    pipeline_comps = init_components(config)

    student_pipeline = init_local_pipeline(pipeline_comps, pipeline_root)
    BeamDagRunner().run(pipeline=student_pipeline)

Writing modules/pipeline.py


# Generate transform.py file

In [7]:
%%writefile {TRANSFORM_MODULE_FILE}
"""
This module defines the feature preprocessing function for the TFX pipeline.

It applies transformations to the input dataset using TensorFlow Transform (TFT).
The transformations include feature scaling, categorical encoding, and data cleaning.
Unused features are removed to optimize model performance.

Main Transformations:
- Standardization: Numeric features are normalized using Z-score normalization.
- One-Hot Encoding: Categorical features are encoded with vocabulary-based one-hot encoding.
- Data Cleaning: Entries with missing values in "Financial Stress" are removed.
"""

import tensorflow as tf
import tensorflow_transform as tft

LABEL_KEY = "Depression"
NUMERIC_FEATURES = ["Academic Pressure", "Age", "CGPA", "Study Satisfaction", "Work/Study Hours"]
CATEGORICAL_FEATURES = ["Dietary Habits", "Family History of Mental Illness", 
                        "Financial Stress", "Gender", "Have you ever had suicidal thoughts ?", 
                        "Sleep Duration"]

UNUSED_FEATURES = ["Job Satisfaction", "Work Pressure", "id", "City", "Profession", "Degree"]

def transformed_name(key):
    """
    Converts feature names by replacing spaces and special characters 
    to make them compatible with TensorFlow Transform.
    """
    return key.replace("/", "_").replace(" ", "_").lower() + "_xf"

def preprocessing_fn(inputs):
    """
    Applies preprocessing to input features.

    Args:
        inputs: A dictionary mapping feature keys to raw tensors.

    Returns:
        outputs: A dictionary mapping feature keys to transformed tensors.
    """
    outputs = {}

    # Remove unused features
    filtered_inputs = {key: tensor for key, tensor in inputs.items() if key not in UNUSED_FEATURES}

    # Remove data where 'Financial Stress' is missing ("?")
    mask = tf.not_equal(filtered_inputs["Financial Stress"], "?")
    clean_inputs = {key: tf.boolean_mask(tensor, mask) for key, tensor in filtered_inputs.items()}

    # Standardize numerical features using Z-score normalization
    for feature in NUMERIC_FEATURES:
        outputs[transformed_name(feature)] = tft.scale_to_z_score(clean_inputs[feature])

    # One-Hot Encoding for categorical features
    for feature in CATEGORICAL_FEATURES:
        vocab_filename = feature.replace(" ", "_").lower() + "_vocab"

        # Apply encoding using the created vocabulary
        indexed = tft.compute_and_apply_vocabulary(
            clean_inputs[feature],
            vocab_filename=vocab_filename,
            num_oov_buckets=1
        )

        vocab_size = tf.cast(tft.experimental.get_vocabulary_size_by_name(vocab_filename) + 1, tf.int32)

        # One-Hot Encoding
        one_hot_encoded = tf.one_hot(indexed, depth=vocab_size, on_value=1.0, off_value=0.0)

        outputs[transformed_name(feature)] = one_hot_encoded

    # Retain the target label without modification
    outputs[LABEL_KEY] = clean_inputs[LABEL_KEY]

    return outputs

Writing modules/transform.py


# Generate tuner.py file

In [8]:
%%writefile {TUNER_MODULE_FILE}
# pylint: disable=no-member
"""
This module defines the hyperparameter tuning process for a deep learning model 
using Keras Tuner and TensorFlow Transform (TFT) in a TFX pipeline.

The tuner applies Bayesian Optimization to search for the best hyperparameters 
for a binary classification model predicting student depression.

Main Components:
- `input_fn()`: Reads and prepares the dataset for training and evaluation.
- `model_builder()`: Defines the neural network architecture with tunable parameters.
- `tuner_fn()`: Configures and executes the Bayesian Optimization tuning process.
"""

from typing import NamedTuple, Dict, Text, Any
import keras_tuner as kt
import tensorflow as tf
from keras import layers
import tensorflow_transform as tft
from keras_tuner.engine import base_tuner
from tfx.components.trainer.fn_args_utils import FnArgs
from modules.transform import transformed_name, LABEL_KEY, NUMERIC_FEATURES, CATEGORICAL_FEATURES

NUM_EPOCHS = 10

TunerFnResult = NamedTuple("TunerFnResult", [
    ("tuner", base_tuner.BaseTuner),
    ("fit_kwargs", Dict[Text, Any]),
])

def gzip_reader_fn(filenames):
    """
    Reads GZIP-compressed TFRecord files and returns a dataset.
    """
    return tf.data.TFRecordDataset(filenames, compression_type="GZIP")

def input_fn(file_pattern, tf_transform_output, num_epochs, batch_size=64):
    """
    Prepares the dataset for training and evaluation.
    """
    feature_spec = tf_transform_output.transformed_feature_spec().copy()
    dataset = tf.data.experimental.make_batched_features_dataset(
        file_pattern=file_pattern,
        batch_size=batch_size,
        features=feature_spec,
        reader=gzip_reader_fn,
        num_epochs=num_epochs,
        label_key=LABEL_KEY)
    return dataset

def build_inputs(tf_transform_output):
    """
    Creates input layers for numeric and categorical features.
    Returns a dictionary of inputs and concatenated feature tensors.
    """
    inputs = {}
    
    # Numeric Features
    numeric_inputs = [tf.keras.Input(shape=(1,), name=transformed_name(f), dtype=tf.float32) for f in NUMERIC_FEATURES]
    for f in NUMERIC_FEATURES:
        inputs[transformed_name(f)] = numeric_inputs[NUMERIC_FEATURES.index(f)]
    
    concat_numeric = layers.concatenate(numeric_inputs)
    
    # Categorical Features
    categorical_inputs = []
    for feature in CATEGORICAL_FEATURES:
        transformed_feature_name = transformed_name(feature)
        vocab_size = tf_transform_output.vocabulary_size_by_name(feature.replace(" ", "_").lower() + "_vocab") + 1
        
        categorical_input = tf.keras.Input(shape=(vocab_size,), name=transformed_feature_name, dtype=tf.float32)
        categorical_inputs.append(categorical_input)
        inputs[transformed_feature_name] = categorical_input
    
    concat_categorical = layers.concatenate(categorical_inputs)

    return inputs, concat_numeric, concat_categorical

def model_builder(hp, tf_transform_output):
    """
    Builds a Keras model with tunable hyperparameters.
    """
    inputs, concat_numeric, concat_categorical = build_inputs(tf_transform_output)
    
    x = layers.Dense(hp.Int("dense_units_1", min_value=32, max_value=128, step=32), activation='relu')(concat_numeric)
    combined = layers.concatenate([x, concat_categorical])
    x = layers.Dense(hp.Int("dense_units_2", min_value=32, max_value=128, step=32), activation='relu')(combined)
    x = layers.Dropout(hp.Float("dropout_rate", min_value=0.1, max_value=0.5, step=0.1))(x)
    outputs = layers.Dense(1, activation='sigmoid')(x)
    
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    model.compile(
        loss='binary_crossentropy',
        optimizer=tf.keras.optimizers.Adam(hp.Choice("learning_rate", values=[1e-2, 1e-3, 1e-4])),
        metrics=[tf.keras.metrics.BinaryAccuracy()]
    )
    return model

def tuner_fn(fn_args: FnArgs):
    """
    Defines the hyperparameter tuning function using Bayesian Optimization.
    """
    tf_transform_output = tft.TFTransformOutput(fn_args.transform_graph_path)
    train_set = input_fn(fn_args.train_files, tf_transform_output, NUM_EPOCHS)
    eval_set = input_fn(fn_args.eval_files, tf_transform_output, NUM_EPOCHS)
    
    tuner = kt.BayesianOptimization(
        hypermodel=lambda hp: model_builder(hp, tf_transform_output),
        objective=kt.Objective('binary_accuracy', direction='max'),
        max_trials=30,
        directory=fn_args.working_dir,
        project_name="bayesian_tuning",
    )
    
    return TunerFnResult(
        tuner=tuner,
        fit_kwargs={
            "x": train_set,
            "validation_data": eval_set,
            "epochs": NUM_EPOCHS,
        },
    )

Writing modules/tuner.py


# Generate trainer.py file

In [9]:
%%writefile {TRAINER_MODULE_FILE}
# pylint: disable=no-member
"""
This module defines the training pipeline for a deep learning model used in a TFX pipeline.

The model predicts depression risk based on various numerical and categorical features.
It utilizes TensorFlow Transform (TFT) for preprocessing and TensorFlow Keras for training.

Main Components:
- `input_fn()`: Reads and prepares the dataset for training and validation.
- `model_builder()`: Defines the neural network architecture.
- `_get_serve_tf_example_fn()`: Creates a serving function for model inference.
- `run_fn()`: Trains the model and exports it for serving.
"""

import os
import tensorflow as tf
import tensorflow_transform as tft
from keras import layers
from tfx.components.trainer.fn_args_utils import FnArgs
from modules.transform import LABEL_KEY
from modules.tuner import input_fn, build_inputs

def model_builder(tf_transform_output):
    """
    Build the deep learning model using transformed features and best hyperparameter from tuning.
    """
    inputs, concat_numeric, concat_categorical = build_inputs(tf_transform_output)

    x = layers.Dense(64, activation='relu')(concat_numeric)
    combined = layers.concatenate([x, concat_categorical])
    x = layers.Dense(64, activation='relu')(combined)
    x = layers.Dense(32, activation='relu')(x)
    outputs = layers.Dense(1, activation='sigmoid')(x)

    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    model.compile(
        loss='binary_crossentropy',
        optimizer=tf.keras.optimizers.Adam(0.01),
        metrics=[tf.keras.metrics.BinaryAccuracy()]
    )
    model.summary()
    return model

def _get_serve_tf_example_fn(model, tf_transform_output):
    model.tft_layer = tf_transform_output.transform_features_layer()
    
    @tf.function
    def serve_tf_examples_fn(serialized_tf_examples):
        feature_spec = tf_transform_output.raw_feature_spec()
        feature_spec.pop(LABEL_KEY)
        
        parsed_features = tf.io.parse_example(serialized_tf_examples, feature_spec)
        transformed_features = model.tft_layer(parsed_features)
        
        return model(transformed_features)
    
    return serve_tf_examples_fn

def run_fn(fn_args: FnArgs):
    """Train and save the model."""
    log_dir = os.path.join(os.path.dirname(fn_args.serving_model_dir), 'logs')
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, update_freq='batch')
    es = tf.keras.callbacks.EarlyStopping(monitor='val_binary_accuracy', mode='max', patience=10)
    mc = tf.keras.callbacks.ModelCheckpoint(
        filepath=os.path.join(fn_args.serving_model_dir, "best_model.keras"),
        monitor='val_binary_accuracy',
        mode='max',
        save_best_only=True
    )
    
    # Load TFT transform graph
    tf_transform_output = tft.TFTransformOutput(fn_args.transform_graph_path)
    
    # Load training and validation datasets
    train_set = input_fn(fn_args.train_files, tf_transform_output, num_epochs=10)
    val_set = input_fn(fn_args.eval_files, tf_transform_output, num_epochs=10)
    
    # Build and train the model
    model = model_builder(tf_transform_output)
    model.fit(
        x=train_set,
        validation_data=val_set,
        callbacks=[tensorboard_callback, es, mc],
        epochs=10)

    signatures = {
    "serving_default": _get_serve_tf_example_fn(
        model, tf_transform_output
    ).get_concrete_function(
        tf.TensorSpec(
            shape=[None], 
            dtype=tf.string, 
            name="examples"
        )
    )}
    
    # Save final model
    tf.saved_model.save(model, fn_args.serving_model_dir, signatures=signatures)

Writing modules/trainer.py


# Run Pipeline

In [12]:
import os
from tfx.orchestration.beam.beam_dag_runner import BeamDagRunner
from tfx.orchestration.pipeline import Pipeline
from modules.pipeline import init_local_pipeline
from modules.components import init_components, ComponentConfig

PIPELINE_NAME = "nandaaryaputra-pipeline"

DATA_ROOT = "/kaggle/input/student-depression-dataset"
TRANSFORM_MODULE_FILE = "modules/transform.py"
TUNER_MODULE_FILE = "modules/tuner.py"
TRAINER_MODULE_FILE = "modules/trainer.py"

OUTPUT_BASE = "output"
serving_model_dir = os.path.join(OUTPUT_BASE, 'serving_model')
pipeline_root = os.path.join(OUTPUT_BASE, PIPELINE_NAME)
metadata_path = os.path.join(pipeline_root, "metadata.sqlite")

config = ComponentConfig(
    data_dir=DATA_ROOT,
    transform_module=TRANSFORM_MODULE_FILE,
    tuner_module=TUNER_MODULE_FILE,
    training_module=TRAINER_MODULE_FILE,
    training_steps=800,
    eval_steps=400,
    serving_model_dir=serving_model_dir
)

components = init_components(config)

pipeline = init_local_pipeline(components, pipeline_root)
BeamDagRunner().run(pipeline)

Trial 30 Complete [00h 02m 26s]
binary_accuracy: 0.8821601271629333

Best binary_accuracy So Far: 0.8836020231246948
Total elapsed time: 01h 13m 53s
Results summary
Results in output/nandaaryaputra-pipeline/Tuner/.system/executor_execution/7/.temp/7/bayesian_tuning
Showing 10 best trials
Objective(name="binary_accuracy", direction="max")

Trial 19 summary
Hyperparameters:
dense_units_1: 128
dense_units_2: 128
dropout_rate: 0.1
learning_rate: 0.001
Score: 0.8836020231246948

Trial 20 summary
Hyperparameters:
dense_units_1: 128
dense_units_2: 128
dropout_rate: 0.1
learning_rate: 0.001
Score: 0.883534848690033

Trial 23 summary
Hyperparameters:
dense_units_1: 128
dense_units_2: 128
dropout_rate: 0.1
learning_rate: 0.001
Score: 0.8821780681610107

Trial 29 summary
Hyperparameters:
dense_units_1: 128
dense_units_2: 128
dropout_rate: 0.1
learning_rate: 0.001
Score: 0.8821601271629333

Trial 15 summary
Hyperparameters:
dense_units_1: 128
dense_units_2: 128
dropout_rate: 0.1
learning_rate: 0.0

# Check code score

In [13]:
!pip install autopep8 pylint

In [14]:
!autopep8 --in-place --aggressive --aggressive modules/components.py modules/pipeline.py modules/transform.py modules/trainer.py modules/tuner.py
!pylint modules/components.py modules/pipeline.py modules/transform.py modules/trainer.py modules/tuner.py


--------------------------------------------------------------------
Your code has been rated at 10.00/10 (previous run: 10.00/10, +0.00)



# Create requirement.txt

In [15]:
!pip freeze >> requirements.txt